In [1]:
from dataset import PBowDataset

clusters = [3, 5, 10]
hm_classes = [1, 2]
data_types = ['train', 'val']
for cluster in clusters:
    for hm in hm_classes:
        for dt in data_types:
            print(cluster, hm, dt)
            _ = PBowDataset(dataset_type=dt, hm_class=hm, nclusters=cluster)

3 1 train

energy_frame.parquet exists. Loading now.
box_frame.parquet exists. Loading now.
force_frame.parquet exists. Loading now.
type_frame.parquet exists. Loading now.
virial_frame.parquet exists. Loading now.

Splitting data into train, val, and test sets...

Creating Rips Frame for 1-Homology...
rips_1_frame.parquet exists. Loading now.

Getting train data...

3 1 val

energy_frame.parquet exists. Loading now.
box_frame.parquet exists. Loading now.
force_frame.parquet exists. Loading now.
type_frame.parquet exists. Loading now.
virial_frame.parquet exists. Loading now.

Splitting data into train, val, and test sets...

Creating Rips Frame for 1-Homology...
rips_1_frame.parquet exists. Loading now.

Getting val data...

3 2 train

energy_frame.parquet exists. Loading now.
box_frame.parquet exists. Loading now.
force_frame.parquet exists. Loading now.
type_frame.parquet exists. Loading now.
virial_frame.parquet exists. Loading now.

Splitting data into train, val, and test sets...

100%|██████████| 108000/108000 [20:01<00:00, 89.92it/s]


5 1 val

energy_frame.parquet exists. Loading now.
box_frame.parquet exists. Loading now.
force_frame.parquet exists. Loading now.
type_frame.parquet exists. Loading now.
virial_frame.parquet exists. Loading now.

Splitting data into train, val, and test sets...

Creating Rips Frame for 1-Homology...
rips_1_frame.parquet exists. Loading now.

Getting val data...

model_data/val-1-5_pbow_embeddings.parquet doesn't exist. Collecting and creating now...


100%|██████████| 14000/14000 [00:13<00:00, 1045.02it/s]


5 2 train

energy_frame.parquet exists. Loading now.
box_frame.parquet exists. Loading now.
force_frame.parquet exists. Loading now.
type_frame.parquet exists. Loading now.
virial_frame.parquet exists. Loading now.

Splitting data into train, val, and test sets...

Creating Rips Frame for 2-Homology...
rips_2_frame.parquet exists. Loading now.

Getting train data...

5 2 val

energy_frame.parquet exists. Loading now.
box_frame.parquet exists. Loading now.
force_frame.parquet exists. Loading now.
type_frame.parquet exists. Loading now.
virial_frame.parquet exists. Loading now.

Splitting data into train, val, and test sets...

Creating Rips Frame for 2-Homology...
rips_2_frame.parquet exists. Loading now.

Getting val data...

10 1 train

energy_frame.parquet exists. Loading now.
box_frame.parquet exists. Loading now.
force_frame.parquet exists. Loading now.
type_frame.parquet exists. Loading now.
virial_frame.parquet exists. Loading now.

Splitting data into train, val, and test sets..

...

100%|██████████| 14000/14000 [00:13<00:00, 1076.38it/s]


10 2 train

energy_frame.parquet exists. Loading now.
box_frame.parquet exists. Loading now.
force_frame.parquet exists. Loading now.
type_frame.parquet exists. Loading now.
virial_frame.parquet exists. Loading now.

Splitting data into train, val, and test sets...

Creating Rips Frame for 2-Homology...
rips_2_frame.parquet exists. Loading now.

Getting train data...

model_data/train-2-10_pbow_embeddings.parquet doesn't exist. Collecting and creating now...


100%|██████████| 108000/108000 [03:37<00:00, 497.59it/s]


10 2 val

energy_frame.parquet exists. Loading now.
box_frame.parquet exists. Loading now.
force_frame.parquet exists. Loading now.
type_frame.parquet exists. Loading now.
virial_frame.parquet exists. Loading now.

Splitting data into train, val, and test sets...

Creating Rips Frame for 2-Homology...
rips_2_frame.parquet exists. Loading now.

Getting val data...

model_data/val-2-10_pbow_embeddings.parquet doesn't exist. Collecting and creating now...


100%|██████████| 14000/14000 [00:08<00:00, 1622.06it/s]


In [16]:
from torch.utils.data import DataLoader

In [18]:
val_loader = DataLoader(pbow_train, batch_size = 100, shuffle=True)
for sample in val_loader:
    print(sample['data'].shape)
    break

torch.Size([100, 5])
